In [2]:
from sgp4.api import Satrec, jday
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [10]:
#import the test tle file
file_path = "tle_active.txt"
with open(file_path, 'r') as file:
    lines = file.readlines()
    
# read test tle file and organize into name, line1 and line2
table = []

for i in range(0, len(lines), 3):
    name = lines[i].strip()
    line_1 = lines[i+1].strip()
    line_2 = lines[i+2].strip()

    table.append([name, line_1, line_2])

columns = ['Name', 'Line 1', 'Line 2']

df = pd.DataFrame(table, columns=columns)
df

,Name,Line 1,Line 2
0,CALSPHERE 1,1 00900U 64063C 25009.89296725 .00000966 0...,2 00900 90.2090 59.8649 0027333 79.0263 331...
1,CALSPHERE 2,1 00902U 64063E 25009.16760729 .00000050 0...,2 00902 90.2243 63.6007 0016938 312.8486 113...
2,LCS 1,1 01361U 65034C 25009.29837725 .00000014 0...,2 01361 32.1465 340.9248 0010070 64.3760 295...
3,TEMPSAT 1,1 01512U 65065E 25009.79003184 .00000068 0...,2 01512 89.9638 213.3307 0067898 278.7099 92...
4,CALSPHERE 4A,1 01520U 65065H 25009.81288413 .00000144 0...,2 01520 89.9354 126.8379 0070779 139.8391 341...
...,...,...,...
10746,STARLINK-32743,1 62480U 24254U 25010.25001157 -.01088896 3...,2 62480 42.9973 259.2015 0001857 262.5092 43...
10747,STARLINK-32753,1 62481U 24254V 25010.25001157 -.01086707 3...,2 62481 42.9973 259.1864 0001466 270.3050 41...
10748,STARLINK-32699,1 62482U 24254W 25010.25001157 -.01098422 3...,2 62482 42.9973 259.1826 0002038 265.0676 48...
10749,THURAYA-4,1 62483U 25001A 25008.40000000 -.00000260 0...,2 62483 26.2230 320.6260 7951270 178.7070 26...


In [11]:
# Calculate xyz, Vx Vy Vz for all the satellites

results = []

# get all the positions and velocities of the satellites
for index, row in df.iterrows():
    name = row['Name']
    line1 = row['Line 1']
    line2 = row['Line 2']

    satellite = Satrec.twoline2rv(line1, line2)

    # Date of when we want to check the position of the satallite, can also loop it to see its trajectory
    year = 2025
    month = 1
    hour = 0
    minute = 0
    second = 0

    # Loop over every day of one month (can change to hours, and can do it over much longer time)
    # Convert the date into julian date
    for day in range(0,1):
        jd, fr = jday(year, month, day, hour, minute, second)

        # Measure the satellite position (r) and velocity (r) at the given date
        e, r, v = satellite.sgp4(jd, fr)
        
        if e == 0:
            results.append({
                "Name": name,
                "Time": f"{year}-{month:02d}-{day:02d} {hour:02d}:{minute:02d}:{second:02d}",
                "Position (km)": list(r),
                "Velocity (km/s)": list(v)
            })
        else:
            print(f"Error propagating {name}: Error code {e}")

columns = ['Name', 'Time', 'Position(km)', 'Velocity(km/s)']
rv_df = pd.DataFrame(results)

rv_df[['Position X (km)', 'Position Y (km)', 'Position Z (km)']] = pd.DataFrame(rv_df['Position (km)'].tolist(), index=rv_df.index)
rv_df[['Velocity X (km/s)', 'Velocity Y (km/s)', 'Velocity Z (km/s)']] = pd.DataFrame(rv_df['Velocity (km/s)'].tolist(), index=rv_df.index)
rv_df = rv_df.drop(columns=['Position (km)', 'Velocity (km/s)'])
rv_df

Error propagating SPACEBEE-6: Error code 1
Error propagating AMICALSAT: Error code 1
Error propagating STARLINK-2028: Error code 6
Error propagating STARLINK-2157: Error code 6
Error propagating STARLINK-2434: Error code 6
Error propagating STARLINK-2349: Error code 6
Error propagating NANOSATC-BR2: Error code 1


,Name,Time,Position X (km),Position Y (km),Position Z (km),Velocity X (km/s),Velocity Y (km/s),Velocity Z (km/s)
0,CALSPHERE 1,2025-01-00 00:00:00,2691.151739,4558.832980,5091.613434,-2.575217,-4.435826,5.294367
1,CALSPHERE 2,2025-01-00 00:00:00,347.208185,628.399924,7401.048079,-3.259261,-6.512702,0.716634
2,LCS 1,2025-01-00 00:00:00,8134.172353,-3446.756135,-2429.410995,3.024485,5.011095,3.046629
3,TEMPSAT 1,2025-01-00 00:00:00,5964.972593,3929.819227,-2388.381961,-1.960150,-1.285583,-6.878444
4,CALSPHERE 4A,2025-01-00 00:00:00,1293.558006,-1736.269835,7147.863392,4.199128,-5.588601,-2.160581
...,...,...,...,...,...,...,...,...
10739,STARLINK-32743,2025-01-00 00:00:00,5585.925878,-3281.634378,554.939383,2.514656,5.190003,5.302163
10740,STARLINK-32753,2025-01-00 00:00:00,5603.168949,-3245.062743,591.930306,2.457860,5.223230,5.296415
10741,STARLINK-32699,2025-01-00 00:00:00,4774.388479,-4353.934956,-699.009517,4.300366,3.876279,5.279048
10742,THURAYA-4,2025-01-00 00:00:00,35653.861527,-44009.198931,-6416.184017,1.417006,0.020929,0.438274


In [7]:
planet_data = {
    "MERCURY": {"distance": 57.96, "mass": 0.330, "inclination": 7.0, "orbital velocity": 47.4},
    "VENUS": {"distance": 108.26, "mass": 4.87, "inclination": 3.4, "orbital velocity": 35.0},
    "EARTH": {"distance": 149.6, "mass": 5.97, "inclination": 0, "orbital velocity": 29.8},
    "MARS": {"distance": 228.0, "mass": 0.642, "inclination": 1.8, "orbital velocity": 24.1},
    "JUPITER": {"distance": 778.5, "mass": 1898, "inclination": 1.3, "orbital velocity": 13.1},
    "SATURN": {"distance": 1432.0, "mass": 568, "inclination": 2.5, "orbital velocity": 9.7},
    "URANUS": {"distance": 2867.0, "mass": 86.8, "inclination": 0.8, "orbital velocity": 6.8},
    "NEPTUNE": {"distance": 4515.0, "mass": 102, "inclination": 1.8, "orbital velocity": 5.4},
    "PLUTO": {"distance": 5906.4, "mass": 0.0130, "inclination": 17.2},
}
# RA DEC data on 21.01.25 at 00h00
ra_dec = {
    'MERCURY': {'RA': (19, 20, 19.92), 'DEC': (-23, 27, 34.0), "distance": 57.96},
    'VENUS': {'RA': (23, 13, 30.71), 'DEC': (-4, 28, 29.0), "distance": 108.26},
    'EARTH': {'RA': (0, 0, 0.00), 'DEC': (0, 0, 0.0), "distance": 149.6},
    'MARS': {'RA': (7, 46, 32.47), 'DEC': (25, 34, 30.5), "distance": 228.0},
    'JUPITER': {'RA': (4, 39, 13.39), 'DEC': (21, 36, 5.4), "distance": 778.5},
    'SATURN': {'RA': (23, 11, 14.87), 'DEC': (-7, 20, 3.9), "distance": 1432.0},
    'URANUS': {'RA': (3, 22, 28.33), 'DEC': (18, 16, 9.7), "distance": 2867.0},
    'NEPTUNE': {'RA': (23, 52, 13.88), 'DEC': (-2, 13, 51.1), "distance": 4515.0},
}


In [8]:
# Calculate the initial coordinates of all the planets on 21.01.2024 at 00h00
initial_coord = {}

for planet, coord in ra_dec.items():
    # Extract RA, DEC, and distance
    ra_h, ra_min, ra_sec = coord['RA']
    dec_h, dec_min, dec_sec = coord['DEC']
    distance = coord['distance']

    # Convert RA and DEC to degrees
    ra_deg = ra_h * 15 + ra_min * 15 / 60 + ra_sec * 15 / 3600
    dec_deg = (abs(dec_h) + dec_min / 60 + dec_sec / 3600) * (-1 if dec_h < 0 else 1)

    # Convert degrees to radians
    ra_rad = np.radians(ra_deg)
    dec_rad = np.radians(dec_deg)

    # Calculate Cartesian coordinates
    x = distance * np.cos(dec_rad) * np.cos(ra_rad)
    y = distance * np.cos(dec_rad) * np.sin(ra_rad)
    z = distance * np.sin(dec_rad)

    initial_coord[planet] = {'x': x, 'y': y, 'z': z}
    
earth_position = initial_coord['EARTH']

# Adjust other planets' positions to be relative to the Sun
for planet in initial_coord:
    if planet == 'EARTH':
        continue  
    # Subtract Earth's position from the planet's position to shift to Sun-centered coordinates
    planet_position = initial_coord[planet]
    sun_centered_x = planet_position['x'] - earth_position['x']
    sun_centered_y = planet_position['y'] - earth_position['y']
    sun_centered_z = planet_position['z'] - earth_position['z']
    
    initial_coord[planet] = {'x': sun_centered_x, 'y': sun_centered_y, 'z': sun_centered_z}


In [9]:
results = []
for planet, data in planet_data.items():
    distance = data['distance']
    inclination = np.radians(data['inclination'])
    initial_position = initial_coord.get(planet, {'x': 0, 'y': 0, 'z': 0})
    
    # Use the initial positions as the starting coordinates
    x, y, z = initial_position['x'], initial_position['y'], initial_position['z']
    
    # Apply rotation matrix for inclination
    rotation_matrix = np.array([
        [1, 0, 0],  
        [0, np.cos(inclination), -np.sin(inclination)],  
        [0, np.sin(inclination), np.cos(inclination)]
    ])

    # Rotate the coordinates
    coords = np.array([x, y, z])
    rotated_coords = np.dot(rotation_matrix, coords)
    
    # Calculate velocity
    orb_vel = data.get('orbital velocity')
    
    # Skip planets without orbital velocity
    if orb_vel is None:
        continue
    theta = np.arctan2(y, x) 
    v_x = -orb_vel * np.sin(theta)
    v_y = orb_vel * np.cos(theta)
    v_z = 0  
    
    velocity = np.array([v_x, v_y, v_z])
    rotated_velocity = np.dot(rotation_matrix, velocity)
    
    results.append({
        'Planet': planet,
        'Position X (kme6)': rotated_coords[0],
        'Position Y (kme6)': rotated_coords[1],
        'Position Z (kme6)': rotated_coords[2],
        'Velocity X (km/s)': rotated_velocity[0],
        'Velocity Y (km/s)': rotated_velocity[1],
        'Velocity Z (km/s)': rotated_velocity[2]
    })
planet_df = pd.DataFrame(results)
planet_df    

,Planet,Position X (kme6),Position Y (kme6),Position Z (kme6),Velocity X (km/s),Velocity Y (km/s),Velocity Z (km/s)
0,MERCURY,-131.342723,-46.752046,-28.987577,16.844993,-43.975577,-5.399522
1,VENUS,-43.882793,-21.203818,-9.720998,15.538951,-31.306256,-1.859934
2,EARTH,149.600000,0.000000,0.000000,0.000000,29.800000,0.000000
3,MARS,-241.799629,180.653133,104.152167,-14.585915,-19.175452,-0.602613
4,JUPITER,100.266280,672.651426,301.942298,-12.959601,1.912295,0.043396
5,SATURN,1238.670072,-291.591895,-195.715031,2.282212,9.418726,0.411230
6,URANUS,1577.781039,2091.544927,928.053643,-5.440550,4.078869,0.056955
7,NEPTUNE,4359.386358,-147.304671,-180.467047,0.189283,5.394019,0.169514
